# 1. Install Dependencies
*Install YOLO and Roboflow packages required for training.*

In [ ]:
!pip install -q -r requirements-train.txt

*Mount google drive (this is for saving logs)*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

*Load required Python modules for training, prediction, and logging.*

In [ ]:
from roboflow import Roboflow
from ultralytics import YOLO
from google.colab import files
from IPython.display import Image, display
import os, glob, csv
from datetime import datetime
import shutil

# 2. Download Dataset from Roboflow
*Authenticate and download your annotated dataset for YOLOv8 training.*

In [ ]:
#  Your Roboflow API key
rf = Roboflow(api_key="***REMOVED")

#  Your workspace slug
#  Your project slug
project = rf.workspace("oceanhazarddetection").project("dataset-used-into-fins")

# Which version of that project to use
version = project.version(1)

# Download it in YOLOv8 format
dataset = version.download("yolov8")

# 3. Set Experiment Metadata & Hyperparameters
*These variables capture both the training setup and the information you’ll
push out to your logs so you can trace exactly how each model was trained and reviewed later. Update these every run.*

In [ ]:
# Experiment log for performance_tracking.csv
# for your logging purposes only
dataset_version = "3"
change_number    = "1"
model_id         = "1"
notes            = "THIS IS A TEST RUN ONLY"
augmentations    = "ADDED FLIP, ROTATE, and HUE ADJUSTMENT"
classes          = "MY CLASS"  #
null_images      = 1462
model_used       = "yolov8s.pt"

# Training Parameters
epochs      = 1
image_size  = 640
data_yaml   = "/content/DATASET-USED-INTO-FINS-1/data.yaml"  # e.g if using colab
batch       = 16

# 4. Train the Model
*Train your YOLOv8 model using your configured hyperparameters.*

In [ ]:
model = YOLO(model_used)
results = model.train(data=data_yaml, epochs=epochs, imgsz=image_size, batch=batch)

# 5. Extract and Save Metrics
*Once training finishes, we automatically pull out the key detection metrics and append them to our central performance log. This ensures every run is recorded with its full context*

In [ ]:
# ---------------------------------------------------------------------------
# 5️⃣ Extract + log metrics *after* training
# ---------------------------------------------------------------------------

# A. Where did YOLO save this run?
trainer     = model.trainer            # trainer object created internally
save_dir    = trainer.save_dir         # e.g. runs/detect/train
folder_name = os.path.basename(save_dir)
args        = trainer.args             # (full CLI args if you still want them)

# B. Metrics from the results object returned by .train()
metrics = {
    "mAP@0.5"     : results.box.map50,
    "mAP@0.5-0.95": results.box.map,
    "precision"   : results.box.p,
    "recall"      : results.box.r,
}

# C. Build one CSV row
import pytz
from datetime import datetime

local_tz  = pytz.timezone("Australia/Brisbane")
timestamp = datetime.now(local_tz).strftime("%d%b_%H-%M")
csv_path  = "performance_tracking.csv"

row = [
    timestamp,
    dataset_version,
    change_number,
    model_id,
    model_used,
    epochs,
    metrics["mAP@0.5"],
    metrics["mAP@0.5-0.95"],
    metrics["precision"],
    metrics["recall"],
    augmentations,
    f"{image_size}x{image_size}",
    classes,
    null_images,
    folder_name,
    notes
]

# D. Append the row (create file & header the first time)
file_exists = os.path.isfile(csv_path)
with open(csv_path, "a", newline="") as f:
    writer = csv.writer(f)
    if not file_exists:
        writer.writerow([
            "Date/Time","Dataset Version","Change #","Model ID","Model Used","Epochs",
            "mAP@0.5","mAP@0.5-0.95","Precision","Recall","Augmentations",
            "Image Size","Classes","Null Images","Run Folder","Notes"
        ])
    writer.writerow(row)

print(f"Metrics logged to {csv_path}")
print(f"Training artifacts are in {save_dir}")

# 6. Backup Model Folder
*Save the full YOLO training folder (weights, graphs, logs) to Drive.*

In [ ]:
named_folder = f"{model_id}_{timestamp}_{folder_name}"
backup_path  = f"/content/drive/MyDrive/yolo_models/{named_folder}"
shutil.copytree(save_dir, backup_path)
print(f" Model folder backed up to: {backup_path}")

# 7. Test the model using your own image
*Upload a test image and run prediction using your trained model.*

In [ ]:
print("\nTry your model out")
uploaded = files.upload()

if uploaded:
    refined_model = YOLO(f"{save_dir}/weights/best.pt")
    for filename in uploaded.keys():
        refined_model.predict(source=filename, save=True, show=False, conf=0.1)

    predict_dir = sorted(glob.glob("runs/detect/predict*"), key=os.path.getmtime)[-1]
    print(f"Predictions are saved in: {predict_dir}")

    for img_file in sorted(glob.glob(f"{predict_dir}/*")):
        display(Image(filename=img_file))
else:
    print("No image uploaded")